In [1]:
using Pkg
Pkg.add("StatsBase")
Pkg.add("ClassImbalance")
Pkg.add("ROCAnalysis")
using Pkg
using DataFrames
using CSV
using Plots
using GLM
using StatsBase
using Lathe
using MLBase
using ClassImbalance
using ROCAnalysis


In [6]:
ENV["COLUMNS"]=1000

1000

In [9]:
df=DataFrame(CSV.File("churn.csv"))
first(df,5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
,Int64,Int64,String,Int64,String,String,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64
1,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,1.01349e5,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.9,1,0,1,1.12543e5,0
3,3,15619304,Onio,502,France,Female,42,8,1.59661e5,3,1,0,1.13932e5,1
4,4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.6,0
5,5,15737888,Mitchell,850,Spain,Female,43,2,1.25511e5,1,1,1,79084.1,0


In [10]:
println(size(df))
describe(df)

(10000, 14)


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,RowNumber,5000.5,1,5000.5,10000,,,Int64
2,CustomerId,1.56909e7,15565701,1.56907e7,15815690,,,Int64
3,Surname,,Abazu,,Zuyeva,2932,,String
4,CreditScore,650.529,350,652.0,850,,,Int64
5,Geography,,France,,Spain,3,,String
6,Gender,,Female,,Male,2,,String
7,Age,38.9218,18,37.0,92,,,Int64
8,Tenure,5.0128,0,5.0,10,,,Int64
9,Balance,76485.9,0.0,97198.5,2.50898e5,,,Float64


In [11]:
names(df)

14-element Array{Symbol,1}:
 :RowNumber
 :CustomerId
 :Surname
 :CreditScore
 :Geography
 :Gender
 :Age
 :Tenure
 :Balance
 :NumOfProducts
 :HasCrCard
 :IsActiveMember
 :EstimatedSalary
 :Exited

In [12]:
Lathe.preprocess.OneHotEncode(df,:Geography)
Lathe.preprocess.OneHotEncode(df,:Gender)
select!(df,Not([:RowNumber,:CustomerId,:Surname,:Geography,:Gender,:Male]))
first(df,5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Spain,Germany,Female
,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64,Bool,Bool,Bool,Bool
1,619,42,2,0.0,1,1,1,1.01349e5,1,1,0,0,1
2,608,41,1,83807.9,1,0,1,1.12543e5,0,0,1,0,1
3,502,42,8,1.59661e5,3,1,0,1.13932e5,1,1,0,0,1
4,699,39,1,0.0,2,0,0,93826.6,0,1,0,0,1
5,850,43,2,1.25511e5,1,1,1,79084.1,0,0,1,0,1


In [15]:
using Lathe.preprocess: TrainTestSplit
train,test=TrainTestSplit(df,.75);

In [16]:
fm = @formula(Exited ~ CreditScore + Age + Tenure +Balance +NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France +Spain)
logit = glm(fm, train, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Binomial{Float64},ProbitLink},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Exited ~ 1 + CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                        Coef.   Std. Error       z  Pr(>|z|)     Lower 95%    Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)      -1.94543      0.16886      -11.52    <1e-29  -2.27639      -1.61447
CreditScore      -0.000341466  0.000187004   -1.83    0.0679  -0.000707988   2.50553e-5
Age               0.0419738    0.0016973     24.73    <1e-99   0.0386471     0.0453004
Tenure           -0.00472278   0.00616724    -0.77    0.4438  -0.0168103     0.00736479
Balance           1.635

In [17]:
prediction = predict(logit,test)

2537-element Array{Union{Missing, Float64},1}:
 0.2258724606811663
 0.16489039410257667
 0.11527827456804528
 0.11039838882182472
 0.06249531565968946
 0.07753887213101966
 0.13234432444275648
 0.05916959410159228
 0.1788428563496759
 0.3995644591658774
 0.2763580761185168
 0.09335821350977436
 0.19600626422838333
 ⋮
 0.08883418154330461
 0.014047738351140518
 0.13247222558960253
 0.5146620424791447
 0.20954984562663728
 0.041785553415771226
 0.09563524685067748
 0.03607148482381811
 0.01462244443259873
 0.13052783365395249
 0.12405582837411777
 0.044342505006766654